<h1>Decision Tree</h1>

In [111]:
library(mlr)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

<h2>1. Dataprep</h2>

In [112]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [113]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [114]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [115]:
drops = c()
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 22

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner <br>
Rpart Parameters: https://www.rdocumentation.org/packages/rpart/versions/4.1-11/topics/rpart.control

In [136]:
cl = "classif.rpart"

List all parameters that can be used in this classifier.
The value must be set in par.vals parameter.

In [137]:
getParamSet(cl)

                   Type len  Def   Constr Req Tunable Trafo
minsplit        integer   -   20 1 to Inf   -    TRUE     -
minbucket       integer   -    - 1 to Inf   -    TRUE     -
cp              numeric   - 0.01   0 to 1   -    TRUE     -
maxcompete      integer   -    4 0 to Inf   -    TRUE     -
maxsurrogate    integer   -    5 0 to Inf   -    TRUE     -
usesurrogate   discrete   -    2    0,1,2   -    TRUE     -
surrogatestyle discrete   -    0      0,1   -    TRUE     -
maxdepth        integer   -   30  1 to 30   -    TRUE     -
xval            integer   -   10 0 to Inf   -   FALSE     -
parms           untyped   -    -        -   -    TRUE     -

In [141]:
learner = makeLearner(cl = cl
                     , predict.type = "prob"
                     , par.vals = list(minsplit = 20
                                      ,cp = 0.01 
                                      ,parms = list(split = 'information') #gini or information
                                      )
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [117]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
                      )

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [118]:
resample = makeResampleDesc(
    method = "CV",
    iters = 10,
    predict = 'both',
    stratify = FALSE
)

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [119]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               ,f1   #f1
               ,ppv  #precision
               ,tpr  #recall
               ,auc  #AUC
               ,gini #Gini
               #,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [120]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE
            )

Resampling: cross-validation
Measures:             mmce.test   acc.test    f1.test     ppv.test    tpr.test    auc.test    gini.test   
[Resample] iter 1:    0.2600000   0.7400000   0.4090909   0.5294118   0.3333333   0.6714866   0.3429731   
[Resample] iter 2:    0.3200000   0.6800000   0.3846154   0.5263158   0.3030303   0.6793306   0.3586612   
[Resample] iter 3:    0.2900000   0.7100000   0.4313725   0.6111111   0.3333333   0.7360923   0.4721845   
[Resample] iter 4:    0.2900000   0.7100000   0.4727273   0.5652174   0.4062500   0.6500460   0.3000919   
[Resample] iter 5:    0.2600000   0.7400000   0.5806452   0.5625000   0.6000000   0.7133333   0.4266667   
[Resample] iter 6:    0.2700000   0.7300000   0.5090909   0.5833333   0.4516129   0.7318841   0.4637681   
[Resample] iter 7:    0.2500000   0.7500000   0.5098039   0.7222222   0.3939394   0.7057892   0.4115785   
[Resample] iter 8:    0.2300000   0.7700000   0.4888889   0.6111111   0.4074074   0.6922882   0.3845764   
[Resampl

<h2>3. Result Analysis</h2>

Train Measures

In [121]:
r$measures.train

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.2166667,0.7833333,0.5877378,0.6950000,0.5091575,0.7687021,0.5374041
2,0.1744444,0.8255556,0.6594360,0.7835052,0.5692884,0.8061162,0.6122323
3,0.1566667,0.8433333,0.6914661,0.8315789,0.5917603,0.8082788,0.6165575
4,0.1800000,0.8200000,0.6772908,0.7264957,0.6343284,0.8108469,0.6216937
5,0.1511111,0.8488889,0.7190083,0.8130841,0.6444444,0.8419959,0.6839918
6,0.1733333,0.8266667,0.6763485,0.7652582,0.6059480,0.8016484,0.6032968
7,0.1966667,0.8033333,0.5629630,0.8260870,0.4269663,0.7667016,0.5334032
8,0.2300000,0.7700000,0.5306122,0.6964286,0.4285714,0.7521835,0.5043670
9,0.1866667,0.8133333,0.6571429,0.7488372,0.5854545,0.7996655,0.5993309
10,0.1822222,0.8177778,0.6289593,0.8128655,0.5129151,0.7823054,0.5646108


Test Measures

In [122]:
r$measures.test

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.26,0.74,0.4090909,0.5294118,0.3333333,0.6714866,0.3429731
2,0.32,0.68,0.3846154,0.5263158,0.3030303,0.6793306,0.3586612
3,0.29,0.71,0.4313725,0.6111111,0.3333333,0.7360923,0.4721845
4,0.29,0.71,0.4727273,0.5652174,0.4062500,0.6500460,0.3000919
5,0.26,0.74,0.5806452,0.5625000,0.6000000,0.7133333,0.4266667
6,0.27,0.73,0.5090909,0.5833333,0.4516129,0.7318841,0.4637681
7,0.25,0.75,0.5098039,0.7222222,0.3939394,0.7057892,0.4115785
8,0.23,0.77,0.4888889,0.6111111,0.4074074,0.6922882,0.3845764
9,0.22,0.78,0.5217391,0.5714286,0.4800000,0.7138667,0.4277333
10,0.27,0.73,0.4255319,0.5555556,0.3448276,0.7047110,0.4094220


Train Aggregated Result

In [123]:
apply(r$measures.train,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.1847778 0.8152222 0.6390965 0.7699140 0.5508834 0.7938444 0.5876888

Test Aggregated Result

In [124]:
apply(r$measures.test,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.2660000 0.7340000 0.4733506 0.5838207 0.4053734 0.6998828 0.3997656

Run Time in seconds

In [125]:
r$runtime

[1] 0.970401

<h2>4. Prediction for new data</h2>

Read the data to predict

In [126]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))

Search for the best model in crossvalidation and use it to score the incoming data

In [127]:
best.model = which.max(r$measures.test$acc)

In [128]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [129]:
pred

Prediction: 1000 observations
predict.type: prob
threshold: 1=0.50,2=0.50
time: 0.01
  truth    prob.1     prob.2 response
1     1 0.9411765 0.05882353        1
2     2 0.2868852 0.71311475        2
3     1 0.8658537 0.13414634        1
4     1 0.2868852 0.71311475        2
5     2 0.2868852 0.71311475        2
6     1 0.8658537 0.13414634        1
... (#rows: 1000, #cols: 4)

Cast result to data.frame to access the prediction

In [130]:
head(as.data.frame(pred))

truth,prob.1,prob.2,response
1,0.9411765,0.05882353,1
2,0.2868852,0.71311475,2
1,0.8658537,0.13414634,1
1,0.2868852,0.71311475,2
2,0.2868852,0.71311475,2
1,0.8658537,0.13414634,1
